# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [23]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d9f3f99075-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Political Magazines explorative search") 

Consider the following exploratory scenario:


>  Investigate the U.S. Magazines which write about politics and their media presence



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P921`    | main subject  | predicate | 
| `wdt:P17`     | country       | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q41298`   | Magazine      | node |
| `wd:Q7163`    | politics      | node |
| `wd:Q30`      | U.S.          | node |
| `wd:Q217305`  | The New Yorker  | node |







Also consider

```
?p wdt:P17 wd:Q30 .
?p wdt:P31/wdt:P279* wd:Q41298  . 
```

is the BGP to retrieve all **type of publications in the U.S.**


The workload should


1. Identify the BGP for obtaining the US magazines that write about politics

2. Compare the number of social media followers and get the top five magazines along with main properties as the place of publication and subfields of work

3. Compare the number of followers with the top three US magazines writing mainly about sports

4. Get the name of notable employees working for The New Yorker and any other political magazine published in the US. Check if these employees have witten any book and if so get the title.  

5. Is any employee of The New Yorker ever been nominated for a prize or award?

In [2]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT(?p)
WHERE { 

?p wdt:P17 wd:Q30 .
?p wdt:P31/wdt:P279* wd:Q41298  . 


} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '5422')]


1

In [27]:
#return pubblications name
#
queryString = """
SELECT ?pubName ?pub
WHERE { 

    ?pub wdt:P17 wd:Q30 .
    ?pub wdt:P31/wdt:P279* wd:Q41298  . 

    ?pub <http://schema.org/name> ?pubName .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pubName', 'Journal of Veterinary Internal Medicine'), ('pub', 'http://www.wikidata.org/entity/Q15762439')]
[('pubName', 'Journal of Veterinary Internal Medicine'), ('pub', 'http://www.wikidata.org/entity/Q15762439')]
[('pubName', 'Molecular Pain'), ('pub', 'http://www.wikidata.org/entity/Q15766244')]
[('pubName', 'Molecular Pain'), ('pub', 'http://www.wikidata.org/entity/Q15766244')]
[('pubName', 'PLOS MEDICINE'), ('pub', 'http://www.wikidata.org/entity/Q1686921')]
[('pubName', 'PLOS MEDICINE'), ('pub', 'http://www.wikidata.org/entity/Q1686921')]
[('pubName', 'Journal of Biomedical Optics'), ('pub', 'http://www.wikidata.org/entity/Q15766132')]
[('pubName', 'Journal of Biomedical Optics'), ('pub', 'http://www.wikidata.org/entity/Q15766132')]
[('pubName', 'Clinical Infectious Diseases'), ('pub', 'http://www.wikidata.org/entity/Q5133764')]
[('pubName', 'Clinical Infectious Diseases'), ('pub', 'http://www.wikidata.org/entity/Q5133764')]
[('pubName', 'Arteriosclerosis, Thrombosis

100

In [7]:
#investigate about predicates and objects connected to publications
queryString = """
SELECT DISTINCT ?objName ?obj ?predName ?pred
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        ?pred ?obj.

    ?pred <http://schema.org/name> ?predName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?objName)
"""

print("Results")
run_query(queryString)

Results
[('objName', 'zoology'), ('obj', 'http://www.wikidata.org/entity/Q431'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('objName', 'zoology'), ('obj', 'http://www.wikidata.org/entity/Q431'), ('predName', 'field of work'), ('pred', 'http://www.wikidata.org/prop/direct/P101')]
[('objName', 'zine'), ('obj', 'http://www.wikidata.org/entity/Q549638'), ('predName', 'instance of'), ('pred', 'http://www.wikidata.org/prop/direct/P31')]
[('objName', 'yearbook'), ('obj', 'http://www.wikidata.org/entity/Q10283140'), ('predName', 'instance of'), ('pred', 'http://www.wikidata.org/prop/direct/P31')]
[('objName', 'worldwide'), ('obj', 'http://www.wikidata.org/entity/Q13780930'), ('predName', 'country'), ('pred', 'http://www.wikidata.org/prop/direct/P17')]
[('objName', 'world-systems theory'), ('obj', 'http://www.wikidata.org/entity/Q1061896'), ('predName', 'field of work'), ('pred', 'http://www.wikidata.org/prop/direct/P101')]
[('objName', 'works copyrigh

3257

In [19]:
#investigate about only predicates of publications
queryString = """
SELECT DISTINCT ?predName ?pred
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        ?pred ?obj.

    ?pred <http://schema.org/name> ?predName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('predName', 'website account on'), ('pred', 'http://www.wikidata.org/prop/direct/P553')]
[('predName', "topic's main category"), ('pred', 'http://www.wikidata.org/prop/direct/P910')]
[('predName', "topic's main Wikimedia portal"), ('pred', 'http://www.wikidata.org/prop/direct/P1151')]
[('predName', 'subsidiary'), ('pred', 'http://www.wikidata.org/prop/direct/P355')]
[('predName', 'subject has role'), ('pred', 'http://www.wikidata.org/prop/direct/P2868')]
[('predName', 'sport'), ('pred', 'http://www.wikidata.org/prop/direct/P641')]
[('predName', 'sponsor'), ('pred', 'http://www.wikidata.org/prop/direct/P859')]
[('predName', 'software engine'), ('pred', 'http://www.wikidata.org/prop/direct/P408')]
[('predName', 'significant person'), ('pred', 'http://www.wikidata.org/prop/direct/P3342')]
[('predName', 'significant event'), ('pred', 'http://www.wikidata.org/prop/direct/P793')]
[('predName', 'said to be the same as'), ('pred', 'http://www.wikidata.org/prop/direct/P460')]
[('predN

101

In [9]:
#investigate about objects of publications
queryString = """
SELECT DISTINCT ?objName ?obj
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        ?pred ?obj.

    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?objName)
"""

print("Results")
run_query(queryString)

Results
[('objName', 'zoology'), ('obj', 'http://www.wikidata.org/entity/Q431')]
[('objName', 'zine'), ('obj', 'http://www.wikidata.org/entity/Q549638')]
[('objName', 'yearbook'), ('obj', 'http://www.wikidata.org/entity/Q10283140')]
[('objName', 'worldwide'), ('obj', 'http://www.wikidata.org/entity/Q13780930')]
[('objName', 'world-systems theory'), ('obj', 'http://www.wikidata.org/entity/Q1061896')]
[('objName', 'works copyrighted by authors'), ('obj', 'http://www.wikidata.org/entity/Q73362150')]
[('objName', 'works copyrighted by learned societies'), ('obj', 'http://www.wikidata.org/entity/Q73362358')]
[('objName', 'works copyrighted by publishers'), ('obj', 'http://www.wikidata.org/entity/Q73362505')]
[('objName', 'woodworking'), ('obj', 'http://www.wikidata.org/entity/Q816871')]
[('objName', "women's sports"), ('obj', 'http://www.wikidata.org/entity/Q920057')]
[('objName', "women's magazine"), ('obj', 'http://www.wikidata.org/entity/Q1059863')]
[('objName', "women's history"), ('obj

2852

Info found:
- Query 1
* [('objName', 'politics'), ('obj', 'http://www.wikidata.org/entity/Q7163'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
* [('objName', 'politics'), ('obj', 'http://www.wikidata.org/entity/Q7163'), ('predName', 'field of work'), ('pred', 'http://www.wikidata.org/prop/direct/P101')]
* [('objName', 'political science'), ('obj', 'http://www.wikidata.org/entity/Q36442'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
- Query 2
* [('predName', 'social media followers'), ('pred', 'http://www.wikidata.org/prop/direct/P8687')]
* [('predName', 'field of work'), ('pred', 'http://www.wikidata.org/prop/direct/P101')]
* [('predName', 'place of publication'), ('pred', 'http://www.wikidata.org/prop/direct/P291')]
- Query 3
* [('objName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q349'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
* [('predName', 'sport'), ('pred', 'http://www.wikidata.org/prop/direct/P641')]
- Query 4
* [('predName', 'place of publication'), ('pred', 'http://www.wikidata.org/prop/direct/P291')]

In [4]:
#investigate about publication that are connected to politics entity
queryString = """
SELECT DISTINCT ?pub ?pubName ?predName ?pred
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        ?pred wd:Q7163.

    ?pred <http://schema.org/name> ?predName .
    ?pub <http://schema.org/name> ?pubName .
} 
ORDER BY DESC( ?objName)
"""

print("Results")
run_query(queryString)

Results
[('pub', 'http://www.wikidata.org/entity/Q217305'), ('pubName', 'The New Yorker'), ('predName', 'field of work'), ('pred', 'http://www.wikidata.org/prop/direct/P101')]
[('pub', 'http://www.wikidata.org/entity/Q6119666'), ('pubName', 'Jacobin'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('pub', 'http://www.wikidata.org/entity/Q7757800'), ('pubName', 'The Port Folio'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('pub', 'http://www.wikidata.org/entity/Q3109740'), ('pubName', 'Politico'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('pub', 'http://www.wikidata.org/entity/Q217305'), ('pubName', 'The New Yorker'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('pub', 'http://www.wikidata.org/entity/Q7747057'), ('pubName', 'The Liberator'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]

7

In [6]:
#investigate about publication that are connected to political science entity
queryString = """
SELECT DISTINCT ?pub ?pubName ?predName ?pred
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        ?pred wd:Q36442.

    ?pred <http://schema.org/name> ?predName .
    ?pub <http://schema.org/name> ?pubName .
} 
ORDER BY DESC( ?objName)
"""

print("Results")
run_query(queryString)

Results
[('pub', 'http://www.wikidata.org/entity/Q6296059'), ('pubName', 'Journal of World-Systems Research'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('pub', 'http://www.wikidata.org/entity/Q7737275'), ('pubName', 'The Good Society'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('pub', 'http://www.wikidata.org/entity/Q6518264'), ('pubName', 'Legislative Studies Quarterly'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('pub', 'http://www.wikidata.org/entity/Q2918282'), ('pubName', 'Dissent'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('pub', 'http://www.wikidata.org/entity/Q3512091'), ('pubName', 'Telos'), ('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('pub', 'http://www.wikidata.org/entity/Q50815250'), ('pubName', 'Research & Politics'), ('predName', 'main subject'), ('pred', 'ht

8

In [87]:
#investigate about predicates and subjects of publications, return the subjects
queryString = """
SELECT DISTINCT ?predName
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?sub ?pred ?pub.
    
    FILTER REGEX(?subName , "politics").
    ?sub <http://schema.org/name> ?subName .
    ?pred <http://schema.org/name> ?predName .
} 
ORDER BY DESC(?predName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('predName', 'published in')]
[('predName', 'described by source')]


2

Investigate Magazine

In [72]:
#investigate the Magazine, return the predicates between Magazine and objects
queryString = """
SELECT DISTINCT ?predName ?pred
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    wd:Q41298 ?pred ?obj.

    ?pred <http://schema.org/name> ?predName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('predName', "topic's main template"), ('pred', 'http://www.wikidata.org/prop/direct/P1424')]
[('predName', "topic's main category"), ('pred', 'http://www.wikidata.org/prop/direct/P910')]
[('predName', "topic's main Wikimedia portal"), ('pred', 'http://www.wikidata.org/prop/direct/P1151')]
[('predName', 'subclass of'), ('pred', 'http://www.wikidata.org/prop/direct/P279')]
[('predName', 'said to be the same as'), ('pred', 'http://www.wikidata.org/prop/direct/P460')]
[('predName', 'properties for this type'), ('pred', 'http://www.wikidata.org/prop/direct/P1963')]
[('predName', 'model item'), ('pred', 'http://www.wikidata.org/prop/direct/P5869')]
[('predName', 'maintained by WikiProject'), ('pred', 'http://www.wikidata.org/prop/direct/P6104')]
[('predName', 'image'), ('pred', 'http://www.wikidata.org/prop/direct/P18')]
[('predName', 'has part'), ('pred', 'http://www.wikidata.org/prop/direct/P527')]
[('predName', 'exact match'), ('pred', 'http://www.wikidata.org/prop/direct/P2888'

38

In [22]:
#investigate the Magazine, return the predicates betwwen subjects and Magazine
queryString = """
SELECT DISTINCT ?predName ?pred
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?sub ?pred wd:Q41298.

    ?pred <http://schema.org/name> ?predName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('predName', 'template has topic'), ('pred', 'http://www.wikidata.org/prop/direct/P1423')]
[('predName', 'subclass of'), ('pred', 'http://www.wikidata.org/prop/direct/P279')]
[('predName', 'said to be the same as'), ('pred', 'http://www.wikidata.org/prop/direct/P460')]
[('predName', 'published in'), ('pred', 'http://www.wikidata.org/prop/direct/P1433')]
[('predName', 'product or material produced'), ('pred', 'http://www.wikidata.org/prop/direct/P1056')]
[('predName', 'part of'), ('pred', 'http://www.wikidata.org/prop/direct/P361')]
[('predName', 'occupation'), ('pred', 'http://www.wikidata.org/prop/direct/P106')]
[('predName', 'newspaper format'), ('pred', 'http://www.wikidata.org/prop/direct/P3912')]
[('predName', 'nature of statement'), ('pred', 'http://www.wikidata.org/prop/direct/P5102')]
[('predName', 'named after'), ('pred', 'http://www.wikidata.org/prop/direct/P138')]
[('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('predName', 'leg

30

In [25]:
#return the subjects connected to US Magazine through 'main subjects'
queryString = """
SELECT DISTINCT ?subName ?sub
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?sub wdt:P921 wd:Q41298.

    ?sub <http://schema.org/name> ?subName .
} 
ORDER BY DESC( ?subName)
"""

print("Results")
run_query(queryString)

Results
[('subName', 'journal 1781-1782'), ('sub', 'http://www.wikidata.org/entity/Q108291445')]
[('subName', 'Zwrot'), ('sub', 'http://www.wikidata.org/entity/Q8075724')]
[('subName', 'WikiProject Academic Journals'), ('sub', 'http://www.wikidata.org/entity/Q21597731')]
[('subName', 'Weekly Bichitra'), ('sub', 'http://www.wikidata.org/entity/Q55637509')]
[('subName', 'Tygodnik Cieszyński'), ('sub', 'http://www.wikidata.org/entity/Q7859856')]
[('subName', 'Titles, columns, news: the description of the contents of the journals, between repertory deficiencies and superficial cataloging'), ('sub', 'http://www.wikidata.org/entity/Q60907379')]
[('subName', 'The misunderstanding between teenagers and reading'), ('sub', 'http://www.wikidata.org/entity/Q84413482')]
[('subName', 'The Italian national digital newspaper library'), ('sub', 'http://www.wikidata.org/entity/Q60907392')]
[('subName', 'The Gold Coast Review Vol. V No. I'), ('sub', 'http://www.wikidata.org/entity/Q108291578')]
[('subNam

63

Subjects of publications

In [10]:
#investigate about predicates and subjects of publications
queryString = """
SELECT DISTINCT ?predName ?pred
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?sub ?pred ?pub.

    ?pred <http://schema.org/name> ?predName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('predName', 'work location'), ('pred', 'http://www.wikidata.org/prop/direct/P937')]
[('predName', 'website account on'), ('pred', 'http://www.wikidata.org/prop/direct/P553')]
[('predName', 'used by'), ('pred', 'http://www.wikidata.org/prop/direct/P1535')]
[('predName', 'target'), ('pred', 'http://www.wikidata.org/prop/direct/P533')]
[('predName', 'subsidiary'), ('pred', 'http://www.wikidata.org/prop/direct/P355')]
[('predName', 'subclass of'), ('pred', 'http://www.wikidata.org/prop/direct/P279')]
[('predName', 'stated in'), ('pred', 'http://www.wikidata.org/prop/direct/P248')]
[('predName', 'sponsor'), ('pred', 'http://www.wikidata.org/prop/direct/P859')]
[('predName', 'signatory'), ('pred', 'http://www.wikidata.org/prop/direct/P1891')]
[('predName', 'said to be the same as'), ('pred', 'http://www.wikidata.org/prop/direct/P460')]
[('predName', 'reviewed by'), ('pred', 'http://www.wikidata.org/prop/direct/P4032')]
[('predName', 'review score by'), ('pred', 'http://www.wikidata

94

Info found:
- Query 4 
* [('predName', 'employer'), ('pred', 'http://www.wikidata.org/prop/direct/P108')]
* [('predName', 'notable work'), ('pred', 'http://www.wikidata.org/prop/direct/P800')]

In [73]:
#investigate about main subject (subject of publication) 
queryString = """
SELECT DISTINCT ?sub ?subName
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?sub wdt:P921 ?pub.

    ?sub <http://schema.org/name> ?subName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q21144537'), ('subName', "PLoS Neglected Tropical Diseases: Two Years of Providing Access to Innovation for the World's Poor … and Counting")]
[('sub', 'http://www.wikidata.org/entity/Q50027688'), ('subName', 'Data sharing and reanalysis of randomized controlled trials in leading biomedical journals with a full data sharing policy: survey of studies published in The BMJ and PLOS Medicine.')]
[('sub', 'http://www.wikidata.org/entity/Q43997847'), ('subName', 'WikiJournal of Medicine, the first Wikipedia-integrated academic journal')]
[('sub', 'http://www.wikidata.org/entity/Q61512110'), ('subName', 'Free access to scientific publications: contrasting the JCI approach to Plan S')]
[('sub', 'http://www.wikidata.org/entity/Q106335971'), ('subName', '2nd International Conference on Computational Linguistics and Intelligent Systems, COLINS 2018, Main Conference, Lviv, Ukraine, June 25-27, 2018')]
[('sub', 'http://www.wikidata.org/entity/Q106245

84

In [39]:
#investigate about predicates and subjects of publications
queryString = """
SELECT DISTINCT ?predName ?pred 
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?sub ?pred ?pub.

    ?pred <http://schema.org/name> ?predName .
    ?sub <http://schema.org/name> ?subName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('predName', 'work location'), ('pred', 'http://www.wikidata.org/prop/direct/P937')]
[('predName', 'website account on'), ('pred', 'http://www.wikidata.org/prop/direct/P553')]
[('predName', 'used by'), ('pred', 'http://www.wikidata.org/prop/direct/P1535')]
[('predName', 'target'), ('pred', 'http://www.wikidata.org/prop/direct/P533')]
[('predName', 'subsidiary'), ('pred', 'http://www.wikidata.org/prop/direct/P355')]
[('predName', 'subclass of'), ('pred', 'http://www.wikidata.org/prop/direct/P279')]
[('predName', 'stated in'), ('pred', 'http://www.wikidata.org/prop/direct/P248')]
[('predName', 'sponsor'), ('pred', 'http://www.wikidata.org/prop/direct/P859')]
[('predName', 'signatory'), ('pred', 'http://www.wikidata.org/prop/direct/P1891')]
[('predName', 'said to be the same as'), ('pred', 'http://www.wikidata.org/prop/direct/P460')]
[('predName', 'reviewed by'), ('pred', 'http://www.wikidata.org/prop/direct/P4032')]
[('predName', 'review score by'), ('pred', 'http://www.wikidata

94

**********************
# Query 1
Identify the BGP for obtaining the US magazines that write about politics

In [39]:
#return the politics publications
queryString = """
SELECT DISTINCT ?pubName ?pub
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P921 wd:Q7163.

    ?pub <http://schema.org/name> ?pubName .
    
} 
ORDER BY DESC( ?objName)
"""

print("Results")
run_query(queryString)

Results
[('pubName', 'The New Yorker'), ('pub', 'http://www.wikidata.org/entity/Q217305')]
[('pubName', 'Politico'), ('pub', 'http://www.wikidata.org/entity/Q3109740')]
[('pubName', 'Commentary'), ('pub', 'http://www.wikidata.org/entity/Q1115290')]
[('pubName', 'Jacobin'), ('pub', 'http://www.wikidata.org/entity/Q6119666')]
[('pubName', 'The Liberator'), ('pub', 'http://www.wikidata.org/entity/Q7747057')]
[('pubName', 'The Port Folio'), ('pub', 'http://www.wikidata.org/entity/Q7757800')]


6

>First of all, I've tried to investigate about the predicates/objects connected to the publication in US.
>
>In this phase, I've found:
>* wd:Q7163 'politics'
>* wd:Q36442 'political science'
>
>and, then, I've return the name of the publications.
>Given the outputs, I've decided to maintain wd:Q7163, because the publications are more specific than the publications returned using wd:Q36442.
>
>Then, I've looked for something useful in the subjects of publications, but I haven't find anything useful.
>
> Later, I've decided to investigate about the predicates/objects/subjects of Magazine entity (given as input in the notebook), but, also in this case, I've not found anything useful.
*************************

In [15]:
#investigate about field of work connected to publications
queryString = """
SELECT DISTINCT ?fName ?predName ?objName
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P101 ?fieldOfWork.
    ?fieldOfWork ?pred ?obj.

    ?pred <http://schema.org/name> ?predName .
    ?fieldOfWork <http://schema.org/name> ?fName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('fName', 'cryptocurrency'), ('predName', 'uses'), ('objName', 'blockchain')]
[('fName', 'cryptocurrency'), ('predName', 'uses'), ('objName', 'directed acyclic graph')]
[('fName', 'cryptocurrency'), ('predName', 'uses'), ('objName', 'cryptocurrency wallet')]
[('fName', 'cryptocurrency'), ('predName', 'uses'), ('objName', 'off-chain governance')]
[('fName', 'cryptocurrency'), ('predName', 'uses'), ('objName', 'cryptographic validation')]
[('fName', 'cryptocurrency'), ('predName', 'uses'), ('objName', 'on-chain governance')]
[('fName', 'business'), ('predName', 'uses'), ('objName', 'social media')]
[('fName', 'fiction'), ('predName', 'uses'), ('objName', 'writing style')]
[('fName', 'technology'), ('predName', 'uses'), ('objName', 'technique')]
[('fName', 'science'), ('predName', 'uses'), ('objName', 'scientific method')]
[('fName', 'motor car'), ('predName', 'uses'), ('objName', 'internal combustion engine')]
[('fName', 'bodybuilding'), ('predName', 'uses'), ('objName', 'weight

1231

In [10]:
#return the social media followers
queryString = """
SELECT DISTINCT ?pubName ?followers
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P8687 ?followers.

    ?pub <http://schema.org/name> ?pubName .
    
} 
ORDER BY DESC( ?followers)
"""

print("Results")
run_query(queryString)

Results
[('pubName', 'Time'), ('followers', '17806837')]
[('pubName', 'Billboard'), ('followers', '10410725')]
[('pubName', 'Wired'), ('followers', '7900000')]
[('pubName', 'GQ'), ('followers', '6280000')]
[('pubName', 'Harvard Business Review'), ('followers', '5663315')]
[('pubName', 'Life'), ('followers', '4231609')]
[('pubName', 'Glamour'), ('followers', '4160000')]
[('pubName', 'Pitchfork'), ('followers', '3310676')]
[('pubName', 'The Hollywood Reporter'), ('followers', '3280311')]
[('pubName', 'Inc.'), ('followers', '2799617')]
[('pubName', 'Game Informer'), ('followers', '2334969')]
[('pubName', 'Fortune'), ('followers', '2291040')]
[('pubName', 'The New York Review of Books'), ('followers', '2289660')]
[('pubName', 'Design Milk'), ('followers', '2218072')]
[('pubName', 'PC Gamer'), ('followers', '2172466')]
[('pubName', 'Who, What, Wear'), ('followers', '2080694')]
[('pubName', 'The Atlantic'), ('followers', '2079799')]
[('pubName', 'New York Magazine'), ('followers', '1814011')

252

In [42]:
#investigate field of works limited to politicians publications
queryString = """
SELECT DISTINCT ?fieldOfWorkName ?predName ?objName
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P101 ?fieldOfWork;
        wdt:P921 wd:Q7163.
    ?fieldOfWork ?pred ?obj.

    ?pred <http://schema.org/name> ?predName .
    ?fieldOfWork <http://schema.org/name> ?fieldOfWorkName .
    ?obj <http://schema.org/name> ?objName .
    
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('fieldOfWorkName', 'fiction'), ('predName', 'uses'), ('objName', 'writing style')]
[('fieldOfWorkName', 'politics'), ('predName', "topic's main template"), ('objName', 'Template:Politics sidebar')]
[('fieldOfWorkName', 'politics'), ('predName', "topic's main category"), ('objName', 'Category:Politics')]
[('fieldOfWorkName', 'popular culture'), ('predName', "topic's main category"), ('objName', 'Category:Popular culture')]
[('fieldOfWorkName', 'cartoon'), ('predName', "topic's main category"), ('objName', 'Category:Cartooning')]
[('fieldOfWorkName', 'politics'), ('predName', "topic's main Wikimedia portal"), ('objName', 'Portal:Politics')]
[('fieldOfWorkName', 'cartoon'), ('predName', 'subclass of'), ('objName', 'drawing')]
[('fieldOfWorkName', 'popular culture'), ('predName', 'subclass of'), ('objName', 'culture')]
[('fieldOfWorkName', 'fiction'), ('predName', 'subclass of'), ('objName', 'literature')]
[('fieldOfWorkName', 'politics'), ('predName', 'subclass of'), ('objName',

44

In [5]:
#return mainsubject
queryString = """
SELECT DISTINCT ?mainSubjectName ?mainSubject 
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P921 ?mainSubject.

    ?mainSubject <http://schema.org/name> ?mainSubjectName .
    
} 
ORDER BY DESC( ?mainSubjectName)
"""

print("Results")
run_query(queryString)

Results
[('mainSubjectName', 'zoology'), ('mainSubject', 'http://www.wikidata.org/entity/Q431')]
[('mainSubjectName', 'woodworking'), ('mainSubject', 'http://www.wikidata.org/entity/Q816871')]
[('mainSubjectName', "women's history"), ('mainSubject', 'http://www.wikidata.org/entity/Q1279400')]
[('mainSubjectName', 'wine'), ('mainSubject', 'http://www.wikidata.org/entity/Q282')]
[('mainSubjectName', 'wildlife conservation'), ('mainSubject', 'http://www.wikidata.org/entity/Q766328')]
[('mainSubjectName', 'white supremacy'), ('mainSubject', 'http://www.wikidata.org/entity/Q698752')]
[('mainSubjectName', 'weird fiction'), ('mainSubject', 'http://www.wikidata.org/entity/Q732782')]
[('mainSubjectName', 'watershed science'), ('mainSubject', 'http://www.wikidata.org/entity/Q105277236')]
[('mainSubjectName', 'watershed management'), ('mainSubject', 'http://www.wikidata.org/entity/Q4387041')]
[('mainSubjectName', 'visual art'), ('mainSubject', 'http://www.wikidata.org/entity/Q36649')]
[('mainSubj

505

[('mainSubjectName', 'sport'), ('mainSubject', 'http://www.wikidata.org/entity/Q349')]

****************
# Query 2
Compare the number of social media followers and get the top five magazines along with main properties as the place of publication and subfields of work

In [52]:
#compare only the social followers, return the publication with the number of followers 
queryString = """
SELECT DISTINCT ?pubName ?followers ?fieldOfWorkName ?placePubName
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P8687 ?followers.

    ?pub <http://schema.org/name> ?pubName .
    
} 
ORDER BY DESC( ?followers)

"""

print("Results")
run_query(queryString)

Results
[('pubName', 'The Economist'), ('followers', '25421540')]
[('pubName', 'Time'), ('followers', '17806837')]
[('pubName', 'Vice'), ('followers', '14500000')]
[('pubName', 'Billboard'), ('followers', '10410725')]
[('pubName', 'First We Feast'), ('followers', '10200000')]
[('pubName', 'Vogue'), ('followers', '9770000')]
[('pubName', 'Veja'), ('followers', '8617912')]
[('pubName', 'Muy Interesante'), ('followers', '8404687')]
[('pubName', 'Wired'), ('followers', '7900000')]
[('pubName', 'National Geographic Traveler'), ('followers', '7543108')]
[('pubName', 'Food & Wine'), ('followers', '6567251')]
[('pubName', 'GQ'), ('followers', '6280000')]
[('pubName', 'Harvard Business Review'), ('followers', '5663315')]
[('pubName', 'The New York Times Book Review'), ('followers', '5580767')]
[('pubName', 'India Today'), ('followers', '5453496')]
[('pubName', 'Filmfare'), ('followers', '5242032')]
[('pubName', 'Vanity Fair'), ('followers', '4859801')]
[('pubName', 'Parents'), ('followers', '47

1766

In [56]:
#return the pubblication name and its followers, field of work and place of publication
#
queryString = """
SELECT DISTINCT ?pubName (SAMPLE(?followers) AS ?follow) (SAMPLE(?fieldOfWorkName) AS ?field) (SAMPLE(?placePubName) AS ?place)
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P101 ?fieldOfWork;
        wdt:P8687 ?followers;
        wdt:P291 ?placePub.

    ?pub <http://schema.org/name> ?pubName .
    ?fieldOfWork <http://schema.org/name> ?fieldOfWorkName .
    ?placePub <http://schema.org/name> ?placePubName .
    
} 
GROUP BY (?pubName)
ORDER BY DESC(?follow)

LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('pubName', 'Time'), ('follow', '17806837'), ('field', 'news'), ('place', 'New York City')]
[('pubName', 'GQ'), ('follow', '6280000'), ('field', 'lifestyle'), ('place', 'New York City')]
[('pubName', 'Scientific American'), ('follow', '3964337'), ('field', 'history'), ('place', 'New York City')]
[('pubName', 'Fortune'), ('follow', '2291040'), ('field', 'finance'), ('place', 'New York City')]
[('pubName', 'Architectural Digest'), ('follow', '2016917'), ('field', 'architecture'), ('place', 'New York City')]


5

>I've found:
>* wdt:P8687 *social media followers*
>* wdt:P101 *field of work*
>* wdt:P291 *place of publication* 
>
> For field of work, I've also tried to check main subjects, but the results are less clear than the results returned using *field of work* predicate.<br>
> In the result, it's possible to see that the top five magazines are based in New York.<br>
> I had to use SAMPLE, because Fortune has two types of *field of work* (returning two different instances) and the result could be inconsistent.
********************

In [57]:
# analyze main subject with sport
queryString = """
SELECT ?pubName 
WHERE { 
    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P921 wd:Q349.
    
    ?pub <http://schema.org/name> ?pubName .
    
} 
ORDER BY DESC( ?fieldOfWorkName)
"""

print("Results")
run_query(queryString)

Results
[('pubName', 'Soccer America')]
[('pubName', 'Soccer America')]


2

In [59]:
#investigate wdt:P641 (sport) predicate  from publications
queryString = """
SELECT DISTINCT ?sportName
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P641 ?sport.

    ?sport <http://schema.org/name> ?sportName .
} 
ORDER BY DESC( ?objName)
"""

print("Results")
run_query(queryString)

Results
[('sportName', 'professional wrestling')]
[('sportName', 'basketball')]
[('sportName', 'chess')]
[('sportName', 'golf')]
[('sportName', 'baseball')]
[('sportName', 'swimming')]
[('sportName', 'athletics')]
[('sportName', 'association football')]


8

In [4]:
#return publication with sport predicates
queryString = """
SELECT ?pubName 
WHERE { 
    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P641 ?sport.
    
    ?pub <http://schema.org/name> ?pubName .
    
} 
ORDER BY DESC( ?fieldOfWorkName)
"""

print("Results")
run_query(queryString)

Results
[('pubName', 'Wrestling Observer Newsletter')]
[('pubName', 'SLAM Magazine')]
[('pubName', 'American Chess Quarterly')]
[('pubName', 'Golf Digest')]
[('pubName', 'WWE Magazine')]
[('pubName', 'Baseball America')]
[('pubName', 'Sporting News')]
[('pubName', 'Track & Field News')]
[('pubName', 'Swimming World Magazine')]
[('pubName', 'Fairways + Greens')]
[('pubName', 'Baseball Digest')]
[('pubName', 'Soccer America')]


12

*************
# Query 3
Compare the number of followers with the top three US magazines writing mainly about sports

In [43]:
#return the top three US magazine writing mainly about sport with followers
queryString = """
SELECT ?pubName ?followers
WHERE { 
    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P641 ?sport;
        wdt:P8687 ?followers.
    
    ?pub <http://schema.org/name> ?pubName .
    
} 
ORDER BY DESC( ?followers)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('pubName', 'Golf Digest'), ('followers', '897948')]
[('pubName', 'Baseball America'), ('followers', '278790')]
[('pubName', 'Sporting News'), ('followers', '225834')]


3

> In the initial queries, I've found two possibilities regarding sport connected to publications:
> * wdt:P921 *main subject* - wd:Q349 *sport*
> * wdt:P641 *sport*
>
> I've checked them and I've found useful information using the second predicates. The first one returns only 'Soccer America'.
*************

In [6]:
#investigate the predicate wdt:P108 (employer)
queryString = """
SELECT DISTINCT ?sub ?subName
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?sub wdt:P108 ?pub.

    ?sub <http://schema.org/name> ?subName .
} 
ORDER BY DESC( ?predName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q4188546'), ('subName', 'Jeffrey M. Drazen')]
[('sub', 'http://www.wikidata.org/entity/Q16977044'), ('subName', 'Walter Lincoln Burrage')]
[('sub', 'http://www.wikidata.org/entity/Q6913696'), ('subName', 'Morris Fishbein')]
[('sub', 'http://www.wikidata.org/entity/Q55979750'), ('subName', 'Pamela W Miller')]
[('sub', 'http://www.wikidata.org/entity/Q42744968'), ('subName', 'Lee Beadling')]
[('sub', 'http://www.wikidata.org/entity/Q7341292'), ('subName', 'Robert A. Schwartz')]
[('sub', 'http://www.wikidata.org/entity/Q74780804'), ('subName', 'Diane Lynne')]
[('sub', 'http://www.wikidata.org/entity/Q90062676'), ('subName', 'Jack F. Conley')]
[('sub', 'http://www.wikidata.org/entity/Q90300035'), ('subName', 'Mary L Berbee')]
[('sub', 'http://www.wikidata.org/entity/Q84045918'), ('subName', 'Dominic Angeloch')]
[('sub', 'http://www.wikidata.org/entity/Q84045613'), ('subName', 'Bettina Engels')]
[('sub', 'http://www.wikidata.org/entity/Q21678

100

In [75]:
#find something in employees for "notable employees"
queryString = """
SELECT DISTINCT ?predName ?pred
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?employees wdt:P108 ?pub;
        ?pred ?obj.
    
    ?pred <http://schema.org/name> ?predName .
} 
ORDER BY DESC( ?predName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('predName', 'ČSFD person ID'), ('pred', 'http://www.wikidata.org/prop/direct/P2605')]
[('predName', 'Étonnants voyageurs participant ID'), ('pred', 'http://www.wikidata.org/prop/direct/P5377')]
[('predName', 'zbMATH author ID'), ('pred', 'http://www.wikidata.org/prop/direct/P1556')]
[('predName', 'writing language'), ('pred', 'http://www.wikidata.org/prop/direct/P6886')]
[('predName', 'work period (start)'), ('pred', 'http://www.wikidata.org/prop/direct/P2031')]
[('predName', 'work period (end)'), ('pred', 'http://www.wikidata.org/prop/direct/P2032')]
[('predName', 'work location'), ('pred', 'http://www.wikidata.org/prop/direct/P937')]
[('predName', 'voice actor'), ('pred', 'http://www.wikidata.org/prop/direct/P725')]
[('predName', 'video'), ('pred', 'http://www.wikidata.org/prop/direct/P10')]
[('predName', 'victory'), ('pred', 'http://www.wikidata.org/prop/direct/P2522')]
[('predName', 'unmarried partner'), ('pred', 'http://www.wikidata.org/prop/direct/P451')]
[('predName', 

100

In [88]:
#find something in employer for "notable employees"
queryString = """
SELECT DISTINCT ?objName ?obj
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?employer wdt:P108 ?pub;
        wdt:P800 ?obj.
    
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?predName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('objName', 'Altmetrics: A manifesto'), ('obj', 'http://www.wikidata.org/entity/Q21678787')]
[('objName', 'SAGE Publications'), ('obj', 'http://www.wikidata.org/entity/Q251266')]
[('objName', "Tributes: Remembering Some of the World's Greatest Wrestlers"), ('obj', 'http://www.wikidata.org/entity/Q43401408')]
[('objName', 'The Skies Belong to Us'), ('obj', 'http://www.wikidata.org/entity/Q17144791')]
[('objName', 'A Tragic Legacy'), ('obj', 'http://www.wikidata.org/entity/Q4660261')]
[('objName', 'How Would a Patriot Act?'), ('obj', 'http://www.wikidata.org/entity/Q5918249')]
[('objName', 'The Fashion Fund'), ('obj', 'http://www.wikidata.org/entity/Q15995064')]
[('objName', 'This Machine Kills Secrets'), ('obj', 'http://www.wikidata.org/entity/Q48816976')]
[('objName', 'No Place to Hide'), ('obj', 'http://www.wikidata.org/entity/Q16978519')]
[('objName', 'NeuroTribes'), ('obj', 'http://www.wikidata.org/entity/Q27841303')]
[('objName', 'Monk Magazine'), ('obj', 'http://www.wikid

64

In [74]:
#investigate notable work P800 connected to publications
queryString = """
SELECT DISTINCT ?subName ?sub
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?sub wdt:P800 ?pub.
    
    ?sub <http://schema.org/name> ?subName .
} 
ORDER BY DESC( ?predName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('subName', 'Association for Information Science and Technology'), ('sub', 'http://www.wikidata.org/entity/Q466807')]
[('subName', 'Jacobus George Robbers'), ('sub', 'http://www.wikidata.org/entity/Q104637399')]
[('subName', 'Sara Miller McCune'), ('sub', 'http://www.wikidata.org/entity/Q7421712')]
[('subName', 'George D. McCune'), ('sub', 'http://www.wikidata.org/entity/Q71048562')]
[('subName', 'Benjamin Apthorp Gould'), ('sub', 'http://www.wikidata.org/entity/Q520915')]
[('subName', 'Gerald L. K. Smith'), ('sub', 'http://www.wikidata.org/entity/Q2802188')]
[('subName', 'Hugh Hefner'), ('sub', 'http://www.wikidata.org/entity/Q194280')]
[('subName', 'Gardner Dozois'), ('sub', 'http://www.wikidata.org/entity/Q720562')]
[('subName', 'Beverly Afaglo'), ('sub', 'http://www.wikidata.org/entity/Q56091581')]
[('subName', 'George H. Scithers'), ('sub', 'http://www.wikidata.org/entity/Q628482')]
[('subName', 'David Levine'), ('sub', 'http://www.wikidata.org/entity/Q141478')]
[('subNam

28

In [44]:
#investigate notable work P800 only for The New Yorker
queryString = """
SELECT DISTINCT ?notablePerson ?notablePersonName
WHERE { 
    ?pub wdt:P17 wd:Q30.
    wd:Q217305 wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 wd:Q217305.
    
    ?notablePerson <http://schema.org/name> ?notablePersonName .
} 
ORDER BY DESC( ?predName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('notablePerson', 'http://www.wikidata.org/entity/Q105667241'), ('notablePersonName', 'Loveis Wise')]


1

In [19]:
#return notable person for every publication in US
queryString = """
SELECT DISTINCT ?notablePerson ?notablePersonName
WHERE { 
    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub.
    
    ?notablePerson <http://schema.org/name> ?notablePersonName .
} 
ORDER BY DESC( ?predName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('notablePerson', 'http://www.wikidata.org/entity/Q466807'), ('notablePersonName', 'Association for Information Science and Technology')]
[('notablePerson', 'http://www.wikidata.org/entity/Q104637399'), ('notablePersonName', 'Jacobus George Robbers')]
[('notablePerson', 'http://www.wikidata.org/entity/Q7421712'), ('notablePersonName', 'Sara Miller McCune')]
[('notablePerson', 'http://www.wikidata.org/entity/Q71048562'), ('notablePersonName', 'George D. McCune')]
[('notablePerson', 'http://www.wikidata.org/entity/Q520915'), ('notablePersonName', 'Benjamin Apthorp Gould')]
[('notablePerson', 'http://www.wikidata.org/entity/Q2802188'), ('notablePersonName', 'Gerald L. K. Smith')]
[('notablePerson', 'http://www.wikidata.org/entity/Q194280'), ('notablePersonName', 'Hugh Hefner')]
[('notablePerson', 'http://www.wikidata.org/entity/Q720562'), ('notablePersonName', 'Gardner Dozois')]
[('notablePerson', 'http://www.wikidata.org/entity/Q56091581'), ('notablePersonName', 'Beverly Afaglo'

28

In [89]:
#try to put together P800 (notable work) and P108 employer
queryString = """
SELECT DISTINCT ?subName ?sub
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?sub wdt:P800 ?pub.
    ?sub wdt:P108 ?pub.
    ?sub <http://schema.org/name> ?subName .
} 
ORDER BY DESC( ?predName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('subName', 'Sara Miller McCune'), ('sub', 'http://www.wikidata.org/entity/Q7421712')]
[('subName', 'George D. McCune'), ('sub', 'http://www.wikidata.org/entity/Q71048562')]


2

Info:
- Query 4
* [('predName', 'significant person'), ('pred', 'http://www.wikidata.org/prop/direct/P3342')]
* [('predName', 'position held'), ('pred', 'http://www.wikidata.org/prop/direct/P39')]

In [3]:
#find something in employees for "notable employees"
queryString = """
SELECT ?objName ?obj
WHERE { 
    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?employees wdt:P108 ?pub;
        wdt:P3342 ?obj.
    
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?objName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('objName', 'Catherine A. Janvier'), ('obj', 'http://www.wikidata.org/entity/Q19647287')]


1

In [7]:
#find something in employees for "notable employees"
queryString = """
SELECT ?objName ?obj
WHERE { 
    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?employees ?pred ?pub;
        wdt:P3342 ?obj.
    
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?objName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('objName', 'Catherine A. Janvier'), ('obj', 'http://www.wikidata.org/entity/Q19647287')]


1

In [13]:
#try to put together P800 (notable work) and P108 employee for The New Yorker
queryString = """
SELECT DISTINCT ?subName ?sub
WHERE { 

    ?pub wdt:P17 wd:Q30.
    wd:Q217305 wdt:P31/wdt:P279* wd:Q41298.
    ?sub wdt:P800 wd:Q217305.
    ?sub wdt:P108 wd:Q217305.
    ?sub <http://schema.org/name> ?subName .
} 
ORDER BY DESC( ?predName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

* [('objName', 'president'), ('obj', 'http://www.wikidata.org/entity/Q1255921')]

In [26]:
#find significant person for politicians magazine
queryString = """
SELECT DISTINCT ?employeesName 
WHERE { 
    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P921 wd:Q7163.
    ?employees wdt:P108 ?pub;
        wdt:P3342 ?significantPerson.
    
    ?employees <http://schema.org/name> ?employeesName .
    
} 
ORDER BY DESC( ?objName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [38]:
#find the role employees for politicians magazine
queryString = """
SELECT DISTINCT ?roleName ?role ?employeesName
WHERE { 
    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P921 wd:Q7163.
    ?employees wdt:P108 ?pub;
        wdt:P39 ?role.
    
    ?role <http://schema.org/name> ?roleName .
    ?employees <http://schema.org/name> ?employeesName .
    
} 
ORDER BY DESC( ?objName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('roleName', 'art director'), ('role', 'http://www.wikidata.org/entity/Q706364'), ('employeesName', 'Françoise Mouly')]
[('roleName', 'Yale Journal of International Affairs'), ('role', 'http://www.wikidata.org/entity/Q8047391'), ('employeesName', 'Ronan Farrow')]
[('roleName', 'staff writer'), ('role', 'http://www.wikidata.org/entity/Q7596574'), ('employeesName', 'D. T. Max')]


3

In [76]:
#find significant person for The New Yorker
queryString = """
SELECT DISTINCT ?signName ?significantPerson
WHERE { 
    wd:Q217305 wdt:P31/wdt:P279* wd:Q41298.
    ?employees wdt:P108 wd:Q217305;
        wdt:P39 ?significantPerson.
    
    ?significantPerson <http://schema.org/name> ?signName .
    
} 
ORDER BY DESC( ?objName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('signName', 'art director'), ('significantPerson', 'http://www.wikidata.org/entity/Q706364')]
[('signName', 'Yale Journal of International Affairs'), ('significantPerson', 'http://www.wikidata.org/entity/Q8047391')]
[('signName', 'staff writer'), ('significantPerson', 'http://www.wikidata.org/entity/Q7596574')]


3

In [90]:
#investigate ?pred connected to notable person
queryString = """
SELECT DISTINCT ?predName ?pred
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
        ?pred ?obj.
        
    ?pred <http://schema.org/name> ?predName .
} 
ORDER BY DESC( ?predName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('predName', 'ČSFD person ID'), ('pred', 'http://www.wikidata.org/prop/direct/P2605')]
[('predName', 'writing language'), ('pred', 'http://www.wikidata.org/prop/direct/P6886')]
[('predName', 'work period (start)'), ('pred', 'http://www.wikidata.org/prop/direct/P2031')]
[('predName', 'work location'), ('pred', 'http://www.wikidata.org/prop/direct/P937')]
[('predName', 'unmarried partner'), ('pred', 'http://www.wikidata.org/prop/direct/P451')]
[('predName', "topic's main category"), ('pred', 'http://www.wikidata.org/prop/direct/P910')]
[('predName', 'spouse'), ('pred', 'http://www.wikidata.org/prop/direct/P26')]
[('predName', 'social media followers'), ('pred', 'http://www.wikidata.org/prop/direct/P8687')]
[('predName', 'significant person'), ('pred', 'http://www.wikidata.org/prop/direct/P3342')]
[('predName', 'signature'), ('pred', 'http://www.wikidata.org/prop/direct/P109')]
[('predName', 'sex or gender'), ('pred', 'http://www.wikidata.org/prop/direct/P21')]
[('predName', 'res

100

In [91]:
#investigate instance of iun order to restrict to the human
queryString = """
SELECT DISTINCT ?objName ?obj
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
        wdt:P31 ?obj.
        
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?objName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('objName', 'open-access publisher'), ('obj', 'http://www.wikidata.org/entity/Q45400320')]
[('objName', 'nonprofit organization'), ('obj', 'http://www.wikidata.org/entity/Q163740')]
[('objName', 'membership organization'), ('obj', 'http://www.wikidata.org/entity/Q6815100')]
[('objName', 'international organization'), ('obj', 'http://www.wikidata.org/entity/Q484652')]
[('objName', 'human'), ('obj', 'http://www.wikidata.org/entity/Q5')]


5

In [32]:
#investigate the notable employees 
queryString = """
SELECT DISTINCT ?objName ?obj
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
        wdt:P108 ?obj.
        
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?objName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('objName', 'Spirou'), ('obj', 'http://www.wikidata.org/entity/Q1431686')]
[('objName', 'School of Visual Arts'), ('obj', 'http://www.wikidata.org/entity/Q1279835')]
[('objName', 'SAGE Publications'), ('obj', 'http://www.wikidata.org/entity/Q251266')]
[('objName', 'Minnesota Public Radio'), ('obj', 'http://www.wikidata.org/entity/Q6868408')]
[('objName', 'Dudley Observatory'), ('obj', 'http://www.wikidata.org/entity/Q1263758')]
[('objName', 'Argentine National Observatory'), ('obj', 'http://www.wikidata.org/entity/Q67665')]


6

In [35]:
#return the notable employees and their pubblication
queryString = """
SELECT DISTINCT ?notablePersonName ?pubName
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
        wdt:P108 ?institution.
        
    ?pub <http://schema.org/name> ?pubName .
    ?notablePerson <http://schema.org/name> ?notablePersonName .
} 
ORDER BY DESC( ?objName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('notablePersonName', 'Sara Miller McCune'), ('pubName', 'SAGE Publications')]
[('notablePersonName', 'George D. McCune'), ('pubName', 'SAGE Publications')]
[('notablePersonName', 'Benjamin Apthorp Gould'), ('pubName', 'The Astronomical Journal')]
[('notablePersonName', 'Joe Kelly'), ('pubName', 'New Moon')]
[('notablePersonName', 'Sergio Aragonés'), ('pubName', 'Mad')]
[('notablePersonName', 'Harvey Kurtzman'), ('pubName', 'Mad')]


6

In [59]:
#return the notable person and their pubblication (only The New Yorker)
queryString = """
SELECT DISTINCT ?notablePersonName ?notablePerson
WHERE { 

    wd:Q217305 wdt:P17 wd:Q30.
    wd:Q217305 wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 wd:Q217305.
        
    ?notablePerson <http://schema.org/name> ?notablePersonName .
} 
ORDER BY DESC( ?objName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('notablePersonName', 'Loveis Wise'), ('notablePerson', 'http://www.wikidata.org/entity/Q105667241')]


1

In [57]:
#return the notable person and their politicians pubblication 
queryString = """
SELECT DISTINCT ?notablePersonName ?notablePerson ?pubName
WHERE { 

    wd:Q217305 wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P921 wd:Q7163.
    ?notablePerson wdt:P800 ?pub.
       
    ?pub <http://schema.org/name> ?pubName.
    ?notablePerson <http://schema.org/name> ?notablePersonName .
} 
ORDER BY DESC( ?objName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('notablePersonName', 'Nathan J. Robinson'), ('notablePerson', 'http://www.wikidata.org/entity/Q63994070'), ('pubName', 'Current Affairs')]
[('notablePersonName', 'Seth Tobocman'), ('notablePerson', 'http://www.wikidata.org/entity/Q7456621'), ('pubName', 'World War 3 Illustrated')]
[('notablePersonName', 'Loveis Wise'), ('notablePerson', 'http://www.wikidata.org/entity/Q105667241'), ('pubName', 'The New Yorker')]
[('notablePersonName', 'Gerry Gable'), ('notablePerson', 'http://www.wikidata.org/entity/Q3511206'), ('pubName', 'Searchlight')]


4

In [64]:
#investigate the plae of pubblications predicates
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE { 

    ?pub wdt:P17 wd:Q30.
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?obj.

    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?objName)
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q575315'), ('objName', 'Wheaton')]
[('obj', 'http://www.wikidata.org/entity/Q61'), ('objName', 'Washington, D.C.')]
[('obj', 'http://www.wikidata.org/entity/Q1528854'), ('objName', 'Vernon')]
[('obj', 'http://www.wikidata.org/entity/Q1814982'), ('objName', 'University of Chicago Press')]
[('obj', 'http://www.wikidata.org/entity/Q30'), ('objName', 'United States of America')]
[('obj', 'http://www.wikidata.org/entity/Q145'), ('objName', 'United Kingdom')]
[('obj', 'http://www.wikidata.org/entity/Q208447'), ('objName', 'Thousand Oaks')]
[('obj', 'http://www.wikidata.org/entity/Q38022'), ('objName', 'St. Louis')]
[('obj', 'http://www.wikidata.org/entity/Q159260'), ('objName', 'Santa Clara')]
[('obj', 'http://www.wikidata.org/entity/Q41211'), ('objName', 'San Juan')]
[('obj', 'http://www.wikidata.org/entity/Q62'), ('objName', 'San Francisco')]
[('obj', 'http://www.wikidata.org/entity/Q28848'), ('objName', 'Saint Paul')]
[('obj', 'http://www.w

60

In [82]:
#investigate the place of pubblications predicates
queryString = """
SELECT DISTINCT ?pred ?predName
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city.
    ?city ?pred wd:Q30.
    
    ?pred <http://schema.org/name> ?predName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P47'), ('predName', 'shares border with')]
[('pred', 'http://www.wikidata.org/prop/direct/P131'), ('predName', 'located in the administrative territorial entity')]
[('pred', 'http://www.wikidata.org/prop/direct/P530'), ('predName', 'diplomatic relation')]
[('pred', 'http://www.wikidata.org/prop/direct/P17'), ('predName', 'country')]
[('pred', 'http://www.wikidata.org/prop/direct/P1376'), ('predName', 'capital of')]


5

In [ ]:
#investigate the plae of pubblications predicates
queryString = """
SELECT DISTINCT ?pub ?pubName
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city.
    ?city wdt:P17 wd:Q30.
    
    ?pub <http://schema.org/name> ?pubName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('pub', 'http://www.wikidata.org/entity/Q1161692'), ('pubName', 'Neurology')]
[('pub', 'http://www.wikidata.org/entity/Q6295713'), ('pubName', 'Journal of Parasitology')]
[('pub', 'http://www.wikidata.org/entity/Q1071725'), ('pubName', 'Nature Immunology')]
[('pub', 'http://www.wikidata.org/entity/Q1091827'), ('pubName', 'Physiological Reviews')]
[('pub', 'http://www.wikidata.org/entity/Q1514304'), ('pubName', 'Journal of Homosexuality')]
[('pub', 'http://www.wikidata.org/entity/Q10695361'), ('pubName', 'The AAPS Journal')]
[('pub', 'http://www.wikidata.org/entity/Q1091719'), ('pubName', 'Journal of Applied Physiology')]
[('pub', 'http://www.wikidata.org/entity/Q1092259'), ('pubName', 'The Journal of General Physiology')]
[('pub', 'http://www.wikidata.org/entity/Q2725967'), ('pubName', 'The Anatomical Record')]
[('pub', 'http://www.wikidata.org/entity/Q4552020'), ('pubName', '16')]
[('pub', 'http://www.wikidata.org/entity/Q2160146'), ('pubName', 'American Journal of Physiology

1104

In [84]:
#return the politician publications which are published in US
queryString = """
SELECT DISTINCT ?pub ?pubName
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921  wd:Q7163.
    ?city wdt:P17 wd:Q30.
      
    ?pub <http://schema.org/name> ?pubName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('pub', 'http://www.wikidata.org/entity/Q217305'), ('pubName', 'The New Yorker')]
[('pub', 'http://www.wikidata.org/entity/Q3109740'), ('pubName', 'Politico')]
[('pub', 'http://www.wikidata.org/entity/Q1115290'), ('pubName', 'Commentary')]
[('pub', 'http://www.wikidata.org/entity/Q3072976'), ('pubName', 'First Things')]
[('pub', 'http://www.wikidata.org/entity/Q6119666'), ('pubName', 'Jacobin')]
[('pub', 'http://www.wikidata.org/entity/Q7753631'), ('pubName', 'The New Inquiry')]
[('pub', 'http://www.wikidata.org/entity/Q7747057'), ('pubName', 'The Liberator')]
[('pub', 'http://www.wikidata.org/entity/Q3101362'), ('pubName', 'George')]
[('pub', 'http://www.wikidata.org/entity/Q15401772'), ('pubName', 'Metropolitan Magazine')]


9

In [74]:
#investigates predicates connected to politicians publication published in US 
queryString = """
SELECT DISTINCT ?pred ?predName
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921  wd:Q7163.
    ?city wdt:P17 wd:Q30.
    ?pub ?pred ?obj.
    
    ?pred <http://schema.org/name> ?predName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P2139'), ('predName', 'total revenue')]
[('pred', 'http://www.wikidata.org/prop/direct/P910'), ('predName', "topic's main category")]
[('pred', 'http://www.wikidata.org/prop/direct/P1476'), ('predName', 'title')]
[('pred', 'http://www.wikidata.org/prop/direct/P6375'), ('predName', 'street address')]
[('pred', 'http://www.wikidata.org/prop/direct/P580'), ('predName', 'start time')]
[('pred', 'http://www.wikidata.org/prop/direct/P8687'), ('predName', 'social media followers')]
[('pred', 'http://www.wikidata.org/prop/direct/P123'), ('predName', 'publisher')]
[('pred', 'http://www.wikidata.org/prop/direct/P2896'), ('predName', 'publication interval')]
[('pred', 'http://www.wikidata.org/prop/direct/P577'), ('predName', 'publication date')]
[('pred', 'http://www.wikidata.org/prop/direct/P1142'), ('predName', 'political ideology')]
[('pred', 'http://www.wikidata.org/prop/direct/P1387'), ('predName', 'political alignment')]
[('pred', 'http

100

In [77]:
#return employees connected to politicians publication published in US 
queryString = """
SELECT DISTINCT ?obj 
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921  wd:Q7163.
    ?city wdt:P17 wd:Q30.
    ?pub wdt:P1128 ?obj.
    
} 
ORDER BY DESC( ?objName)
"""

print("Results")
run_query(queryString)

Results
[('obj', '500')]


1

In [81]:
#return predicates (outcoming) connected to politicians publication published in US 
queryString = """
SELECT DISTINCT  ?predName ?pred
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921  wd:Q7163.
    ?city wdt:P17 wd:Q30.
    ?sub ?pred ?pub.
    
    ?pred <http://schema.org/name> ?predName .
    ?sub <http://schema.org/name> ?subName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('predName', 'used by'), ('pred', 'http://www.wikidata.org/prop/direct/P1535')]
[('predName', 'subclass of'), ('pred', 'http://www.wikidata.org/prop/direct/P279')]
[('predName', 'publisher'), ('pred', 'http://www.wikidata.org/prop/direct/P123')]
[('predName', 'published in'), ('pred', 'http://www.wikidata.org/prop/direct/P1433')]
[('predName', 'owner of'), ('pred', 'http://www.wikidata.org/prop/direct/P1830')]
[('predName', 'owned by'), ('pred', 'http://www.wikidata.org/prop/direct/P127')]
[('predName', 'occupation'), ('pred', 'http://www.wikidata.org/prop/direct/P106')]
[('predName', 'notable work'), ('pred', 'http://www.wikidata.org/prop/direct/P800')]
[('predName', 'member of'), ('pred', 'http://www.wikidata.org/prop/direct/P463')]
[('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('predName', 'influenced by'), ('pred', 'http://www.wikidata.org/prop/direct/P737')]
[('predName', 'has written for'), ('pred', 'http://www.wikidata.org/prop/di

23

In [82]:
#return predicates (outcoming) connected to politicians publication published in US 
queryString = """
SELECT DISTINCT  ?predName ?pred
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921  wd:Q7163.
    ?city wdt:P17 wd:Q30.
    ?employees wdt:P108 ?pub.
    ?employees ?pred ?obj.
    
    ?pred <http://schema.org/name> ?predName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('predName', 'writing language'), ('pred', 'http://www.wikidata.org/prop/direct/P6886')]
[('predName', 'unmarried partner'), ('pred', 'http://www.wikidata.org/prop/direct/P451')]
[('predName', "topic's main category"), ('pred', 'http://www.wikidata.org/prop/direct/P910')]
[('predName', 'time period'), ('pred', 'http://www.wikidata.org/prop/direct/P2348')]
[('predName', 'student of'), ('pred', 'http://www.wikidata.org/prop/direct/P1066')]
[('predName', 'spouse'), ('pred', 'http://www.wikidata.org/prop/direct/P26')]
[('predName', 'sport'), ('pred', 'http://www.wikidata.org/prop/direct/P641')]
[('predName', 'sibling'), ('pred', 'http://www.wikidata.org/prop/direct/P3373')]
[('predName', 'sexual orientation'), ('pred', 'http://www.wikidata.org/prop/direct/P91')]
[('predName', 'sex or gender'), ('pred', 'http://www.wikidata.org/prop/direct/P21')]
[('predName', 'residence'), ('pred', 'http://www.wikidata.org/prop/direct/P551')]
[('predName', 'represented by'), ('pred', 'http://www.w

60

In [92]:
#return the things in which the employees connected to politicians publication published in US is nominated for
queryString = """
SELECT DISTINCT  ?objName ?obj
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921  wd:Q7163.
    ?city wdt:P17 wd:Q30.
    ?employees wdt:P108 ?pub.
    ?employees wdt:P1411 ?obj.
    
  
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('objName', 'Pulitzer Prize for General Non-Fiction'), ('obj', 'http://www.wikidata.org/entity/Q784589')]
[('objName', 'Nobel Prize in Literature'), ('obj', 'http://www.wikidata.org/entity/Q37922')]
[('objName', 'National Book Award for Nonfiction'), ('obj', 'http://www.wikidata.org/entity/Q3873147')]


3

In [91]:
#return the notable employees connected to politicians publication published in US 
queryString = """
SELECT DISTINCT ?employeesName ?employees ?pubName
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921 wd:Q7163.
    ?city wdt:P17 wd:Q30.
    ?employees wdt:P108 ?pub.
    ?employees wdt:P800 ?obj.
    
   
    ?employees <http://schema.org/name> ?employeesName .
    ?pub  <http://schema.org/name> ?pubName.
} 
ORDER BY DESC( ?employeesName)
"""

print("Results")
run_query(queryString)

Results
[('employeesName', 'William Finnegan'), ('employees', 'http://www.wikidata.org/entity/Q8009057'), ('pubName', 'The New Yorker')]
[('employeesName', 'Talia Lavin'), ('employees', 'http://www.wikidata.org/entity/Q106727554'), ('pubName', 'The New Yorker')]
[('employeesName', 'Steve Coll'), ('employees', 'http://www.wikidata.org/entity/Q128771'), ('pubName', 'The New Yorker')]
[('employeesName', 'Saul Steinberg'), ('employees', 'http://www.wikidata.org/entity/Q432856'), ('pubName', 'The New Yorker')]
[('employeesName', 'Lewis Mumford'), ('employees', 'http://www.wikidata.org/entity/Q469888'), ('pubName', 'The New Yorker')]
[('employeesName', 'Lawrence Wright'), ('employees', 'http://www.wikidata.org/entity/Q353893'), ('pubName', 'The New Yorker')]
[('employeesName', 'Larissa MacFarquhar'), ('employees', 'http://www.wikidata.org/entity/Q54766027'), ('pubName', 'The New Yorker')]
[('employeesName', 'Jonathan Schell'), ('employees', 'http://www.wikidata.org/entity/Q1703300'), ('pubNa

15

In [90]:
#return the notable employees connected to politicians publication published in US (only The New Yorker)
queryString = """
SELECT DISTINCT ?employeesName ?employees
WHERE { 
    wd:Q217305 wdt:P31/wdt:P279* wd:Q41298.
    ?employees wdt:P108 wd:Q217305.
    ?employees wdt:P800 ?institution.
    
   
    ?employees <http://schema.org/name> ?employeesName .
} 
ORDER BY DESC( ?employeesName)
"""

print("Results")
run_query(queryString)

Results
[('employeesName', 'William Finnegan'), ('employees', 'http://www.wikidata.org/entity/Q8009057')]
[('employeesName', 'Talia Lavin'), ('employees', 'http://www.wikidata.org/entity/Q106727554')]
[('employeesName', 'Steve Coll'), ('employees', 'http://www.wikidata.org/entity/Q128771')]
[('employeesName', 'Saul Steinberg'), ('employees', 'http://www.wikidata.org/entity/Q432856')]
[('employeesName', 'Lewis Mumford'), ('employees', 'http://www.wikidata.org/entity/Q469888')]
[('employeesName', 'Lawrence Wright'), ('employees', 'http://www.wikidata.org/entity/Q353893')]
[('employeesName', 'Larissa MacFarquhar'), ('employees', 'http://www.wikidata.org/entity/Q54766027')]
[('employeesName', 'Jonathan Schell'), ('employees', 'http://www.wikidata.org/entity/Q1703300')]
[('employeesName', 'Jillian Tamaki'), ('employees', 'http://www.wikidata.org/entity/Q16221210')]
[('employeesName', 'Jean-Jacques Sempé'), ('employees', 'http://www.wikidata.org/entity/Q354371')]
[('employeesName', 'Hannah A

15

In [95]:
#return the employees connected to politicians publication published in US that have been nominated for something
queryString = """
SELECT DISTINCT  ?employeesName ?employees ?pubName
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921  wd:Q7163.
    ?city wdt:P17 wd:Q30.
    ?employees wdt:P108 ?pub.
    ?employees wdt:P1411 ?award.
    
  
    ?employees <http://schema.org/name> ?employeesName .
    ?pub  <http://schema.org/name> ?pubName.
} 
ORDER BY DESC( ?employeesName)
"""

print("Results")
run_query(queryString)

Results
[('employeesName', 'Steve Coll'), ('employees', 'http://www.wikidata.org/entity/Q128771'), ('pubName', 'The New Yorker')]
[('employeesName', 'Lewis Mumford'), ('employees', 'http://www.wikidata.org/entity/Q469888'), ('pubName', 'The New Yorker')]
[('employeesName', 'Lawrence Wright'), ('employees', 'http://www.wikidata.org/entity/Q353893'), ('pubName', 'The New Yorker')]
[('employeesName', 'Hannah Arendt'), ('employees', 'http://www.wikidata.org/entity/Q60025'), ('pubName', 'The New Yorker')]
[('employeesName', 'Edmund Wilson'), ('employees', 'http://www.wikidata.org/entity/Q704931'), ('pubName', 'The New Yorker')]


5

In [108]:
#return the predicates connected to employees connected to politicians publication published in US that have been nominated for something
queryString = """
SELECT DISTINCT ?predName ?pred
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921  wd:Q7163.
    ?city wdt:P17 wd:Q30.
    ?employees wdt:P108 ?pub.
    ?sub ?pred ?employees.
    
  
    ?pred <http://schema.org/name> ?predName .
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('predName', 'winner'), ('pred', 'http://www.wikidata.org/prop/direct/P1346')]
[('predName', 'unmarried partner'), ('pred', 'http://www.wikidata.org/prop/direct/P451')]
[('predName', 'student'), ('pred', 'http://www.wikidata.org/prop/direct/P802')]
[('predName', 'spouse'), ('pred', 'http://www.wikidata.org/prop/direct/P26')]
[('predName', 'significant person'), ('pred', 'http://www.wikidata.org/prop/direct/P3342')]
[('predName', 'signatory'), ('pred', 'http://www.wikidata.org/prop/direct/P1891')]
[('predName', 'sibling'), ('pred', 'http://www.wikidata.org/prop/direct/P3373')]
[('predName', 'screenwriter'), ('pred', 'http://www.wikidata.org/prop/direct/P58')]
[('predName', 'relative'), ('pred', 'http://www.wikidata.org/prop/direct/P1038')]
[('predName', 'participant'), ('pred', 'http://www.wikidata.org/prop/direct/P710')]
[('predName', 'named after'), ('pred', 'http://www.wikidata.org/prop/direct/P138')]
[('predName', 'mother'), ('pred', 'http://www.wikidata.org/prop/direct/P25

39

In [110]:
#investigate creator predicates connected to employees connected to politicians publication published in US that have been nominated for something
queryString = """
SELECT DISTINCT ?sub ?subName
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921  wd:Q7163.
    ?city wdt:P17 wd:Q30.
    ?employees wdt:P108 ?pub;
        wdt:P800 ?notableWork.
    ?sub wdt:P170 ?employees.
    
    ?sub <http://schema.org/name> ?subName .
  
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q7928780'), ('subName', 'View of the World from 9th Avenue')]
[('sub', 'http://www.wikidata.org/entity/Q60475117'), ('subName', 'The Model Exhibit')]
[('sub', 'http://www.wikidata.org/entity/Q20504847'), ('subName', 'Guaranty')]
[('sub', 'http://www.wikidata.org/entity/Q3225493'), ('subName', 'Le Petit Nicolas')]
[('sub', 'http://www.wikidata.org/entity/Q28134790'), ('subName', 'Mekong Delta')]
[('sub', 'http://www.wikidata.org/entity/Q76357305'), ('subName', 'Legs')]
[('sub', 'http://www.wikidata.org/entity/Q77259899'), ('subName', 'Portrait of M')]
[('sub', 'http://www.wikidata.org/entity/Q105890735'), ('subName', '2015 TCAF Butter Tart Map')]
[('sub', 'http://www.wikidata.org/entity/Q573772'), ('subName', 'banality of evil')]
[('sub', 'http://www.wikidata.org/entity/Q25095190'), ('subName', 'Nuits de la Fondation Maeght')]
[('sub', 'http://www.wikidata.org/entity/Q64518749'), ('subName', 'Life of Millet')]
[('sub', 'http://www.wikidat

21

In [111]:
#investigate creation predicates connected to employees connected to politicians publication published in US that have been nominated for something
queryString = """
SELECT DISTINCT ?predName ?objName
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921  wd:Q7163.
    ?city wdt:P17 wd:Q30.
    ?employees wdt:P108 ?pub;
        wdt:P800 ?notableWork.
    ?creation wdt:P170 ?employees.
    ?creation ?pred ?obj.
    
    
    ?pred <http://schema.org/name> ?predName .
    ?creation <http://schema.org/name> ?creationName .
    ?obj <http://schema.org/name> ?objName .
  
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('predName', 'significant event'), ('objName', 'Toronto Comic Arts Festival')]
[('predName', 'record label'), ('objName', 'Shandar')]
[('predName', 'publisher'), ('objName', 'Instagram')]
[('predName', 'performer'), ('objName', 'Sun Ra')]
[('predName', 'original language of film or TV show'), ('objName', 'German')]
[('predName', 'original broadcaster'), ('objName', 'M6')]
[('predName', 'main subject'), ('objName', 'Toronto')]
[('predName', 'main subject'), ('objName', 'Toronto Comic Arts Festival')]
[('predName', 'main subject'), ('objName', 'butter tart')]
[('predName', 'made from material'), ('objName', 'wove paper')]
[('predName', 'location of the point of view'), ('objName', 'Ninth Avenue')]
[('predName', 'location of creation'), ('objName', 'United States of America')]
[('predName', 'location'), ('objName', 'Smithsonian American Art Museum')]
[('predName', 'location'), ('objName', 'Metropolitan Museum of Art')]
[('predName', 'location'), ('objName', 'National Gallery of A

60

In [117]:
#return the name of the creation (book) connected to employees connected to politicians publication published in US that have been nominated for something
queryString = """
SELECT DISTINCT ?creationName
WHERE { 
    ?pub wdt:P31/wdt:P279* wd:Q41298;
        wdt:P291 ?city;
        wdt:P921  wd:Q7163.
    ?city wdt:P17 wd:Q30.
    ?employees wdt:P108 ?pub;
        wdt:P800 ?notableWork.
    ?creation wdt:P170 ?employees.
    ?creation wdt:P31 ?field.
    ?field wdt:P279 wd:Q571.
    
    ?creation <http://schema.org/name> ?creationName .
  
} 
ORDER BY DESC( ?predName)
"""

print("Results")
run_query(queryString)

Results
[('creationName', '2015 TCAF Butter Tart Map')]


1

In [65]:
#return the notable employees connected to politicians publication published in US (+ The New Yorker)
#First: only The New Yorker
#Second: politicians publication published in US
#NB: I've considered the employees that have done some notable works as notable employees
queryString = """
SELECT DISTINCT ?objName ?obj ?predName
WHERE { 
    {
        wd:Q217305 wdt:P31/wdt:P279* wd:Q41298.
        ?employees wdt:P108 wd:Q217305;
                 wdt:P800 ?notableWork.
    }
    UNION
    {
        ?pub wdt:P17 wd:Q30.
        ?pub wdt:P31/wdt:P279* wd:Q41298;
                wdt:P921 wd:Q7163.
        ?employees wdt:P108 ?pub;
             wdt:P800 ?notableWork.              
    }
    
    ?notableWork ?pred ?obj.
    
    ?obj <http://schema.org/name> ?objName .
    ?pred <http://schema.org/name> ?predName .
} 
ORDER BY ( ?objName)
"""

print("Results")
run_query(queryString)

Results
[('objName', 'A Life in Pieces'), ('obj', 'http://www.wikidata.org/entity/Q4657722'), ('predName', 'different from')]
[('objName', 'Adolf Eichmann'), ('obj', 'http://www.wikidata.org/entity/Q28085'), ('predName', 'main subject')]
[('objName', 'Afghanistan'), ('obj', 'http://www.wikidata.org/entity/Q889'), ('predName', 'main subject')]
[('objName', 'Al-Qaeda'), ('obj', 'http://www.wikidata.org/entity/Q34490'), ('predName', 'main subject')]
[('objName', 'Alfred A. Knopf'), ('obj', 'http://www.wikidata.org/entity/Q1431868'), ('predName', 'publisher')]
[('objName', 'Allen Lane'), ('obj', 'http://www.wikidata.org/entity/Q1263483'), ('predName', 'publisher')]
[('objName', 'American English'), ('obj', 'http://www.wikidata.org/entity/Q7976'), ('predName', 'language of work or name')]
[('objName', 'Binjamin Wilkomirski'), ('obj', 'http://www.wikidata.org/entity/Q124437'), ('predName', 'main subject')]
[('objName', 'Blake Eskin'), ('obj', 'http://www.wikidata.org/entity/Q66433892'), ('pr

104

*********** 
restart the workflow for the 4th query

In [ ]:
#return the publication and the notable person (restrict to human)
queryString = """
SELECT DISTINCT ?npName ?notableWork
WHERE { 

    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5.
    
    ?notablePerson <http://schema.org/name> ?npName .
} 
ORDER BY ( ?objName)
"""

print("Results")
run_query(queryString)

Results
[('npName', 'Pavel Vasilyevich Sergeyev')]
[('npName', 'Eugène Minkowski')]
[('npName', 'Misbah Khan')]
[('npName', 'Arthur Tansley')]
[('npName', 'Reginald Crundall Punnett')]
[('npName', 'Matilda Smith')]
[('npName', 'Miss Adams')]
[('npName', 'Peter Gruss')]
[('npName', 'Elizabeth Harrison')]
[('npName', 'M. Hart')]
[('npName', 'Basil Lanneau Gildersleeve')]
[('npName', 'Van Ikin')]
[('npName', 'Jacobus George Robbers')]
[('npName', 'Sara Miller McCune')]
[('npName', 'Carla Joosten')]
[('npName', 'George D. McCune')]
[('npName', 'Matariki Williams')]
[('npName', 'Marion Graham Elkins')]
[('npName', 'Edna Louise Ferry')]
[('npName', 'Emma L. Wardell')]
[('npName', 'Édouard Herriot')]
[('npName', 'Ernst Kornemann')]
[('npName', 'Jacques Quatreboeufs')]
[('npName', 'François Widmer')]
[('npName', 'Charles Antoni')]
[('npName', 'Albert Chapon')]
[('npName', 'Heinrich Weingartner')]
[('npName', 'Abelardo Lopez-Palacios')]
[('npName', 'Pierre Drieu La Rochelle')]
[('npName', 'Clau

263

In [99]:
#investigate the notable work of the notable person
queryString = """
SELECT DISTINCT ?pred ?pName
WHERE { 

    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5;
                ?pred ?obj.
    
    ?pred <http://schema.org/name> ?pName .
} 
ORDER BY DESC( ?pName)
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P2605'), ('pName', 'ČSFD person ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P5377'), ('pName', 'Étonnants voyageurs participant ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P1556'), ('pName', 'zbMATH author ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P8720'), ('pName', 'xHamster performer ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P6886'), ('pName', 'writing language')]
[('pred', 'http://www.wikidata.org/prop/direct/P2031'), ('pName', 'work period (start)')]
[('pred', 'http://www.wikidata.org/prop/direct/P2032'), ('pName', 'work period (end)')]
[('pred', 'http://www.wikidata.org/prop/direct/P937'), ('pName', 'work location')]
[('pred', 'http://www.wikidata.org/prop/direct/P553'), ('pName', 'website account on')]
[('pred', 'http://www.wikidata.org/prop/direct/P10'), ('pName', 'video')]
[('pred', 'http://www.wikidata.org/prop/direct/P9183'), ('pName', 'uta-net.com artist ID')]
[('pred', 'http://www

616

In [100]:
#investigate the notable work of the notable person (restrict the predicate to wdt:P800)
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE { 

    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5;
                wdt:P800 ?obj.
    
    ?obj <http://schema.org/name> ?oName .
} 
ORDER BY DESC( ?pName)
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q7431607'), ('oName', 'Schizophrenia Research')]
[('obj', 'http://www.wikidata.org/entity/Q15765695'), ('oName', 'Journal of School Health')]
[('obj', 'http://www.wikidata.org/entity/Q1943386'), ('oName', 'Molecular Pharmacology')]
[('obj', 'http://www.wikidata.org/entity/Q5133787'), ('oName', 'Clinical Pediatrics')]
[('obj', 'http://www.wikidata.org/entity/Q13548580'), ('oName', 'New Phytologist')]
[('obj', 'http://www.wikidata.org/entity/Q14745598'), ('oName', "Edwards's Botanical Register")]
[('obj', 'http://www.wikidata.org/entity/Q3025402'), ('oName', 'Developmental Biology')]
[('obj', 'http://www.wikidata.org/entity/Q2269599'), ('oName', "Curtis's Botanical Magazine")]
[('obj', 'http://www.wikidata.org/entity/Q975661'), ('oName', 'Journal of Genetics')]
[('obj', 'http://www.wikidata.org/entity/Q912644'), ('oName', 'punnett square')]
[('obj', 'http://www.wikidata.org/entity/Q47158174'), ('oName', 'Report on the scientific results of

341

In [115]:
#investigate the notable work if there is something about book
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE { 

    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5;
                wdt:P800 ?notableWork.
    ?notableWork wdt:P31 ?obj.
    
    ?obj <http://schema.org/name> ?oName .
} 
ORDER BY ( ?oName)
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q73365499'), ('oName', 'APC-free journal')]
[('obj', 'http://www.wikidata.org/entity/Q503354'), ('oName', 'Christmas carol')]
[('obj', 'http://www.wikidata.org/entity/Q6107132'), ('oName', 'Magazine about comics')]
[('obj', 'http://www.wikidata.org/entity/Q12408563'), ('oName', 'Rabbinical Periodical')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('oName', 'Summer Olympic Games')]
[('obj', 'http://www.wikidata.org/entity/Q4167410'), ('oName', 'Wikimedia disambiguation page')]
[('obj', 'http://www.wikidata.org/entity/Q737498'), ('oName', 'academic journal')]
[('obj', 'http://www.wikidata.org/entity/Q856234'), ('oName', 'academic library')]
[('obj', 'http://www.wikidata.org/entity/Q482994'), ('oName', 'album')]
[('obj', 'http://www.wikidata.org/entity/Q171341'), ('oName', 'almanac')]
[('obj', 'http://www.wikidata.org/entity/Q567357'), ('oName', 'annual publication')]
[('obj', 'http://www.wikidata.org/entity/Q21834496'), ('oName', '

131

In [117]:
#investigate the notable work if there is something about book
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE { 

    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5;
                wdt:P800 ?notableWork.
    ?notableWork wdt:279 ?obj.
    
    ?obj <http://schema.org/name> ?oName .
} 
ORDER BY ( ?oName)
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [116]:
#investigate the notable work if there is something about book
queryString = """
SELECT DISTINCT ?notableWork ?nwName
WHERE { 

    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5;
                wdt:P800 ?notableWork.
    ?notableWork wdt:P31 wd:Q277759.
    
    ?notableWork <http://schema.org/name> ?nwName .
} 
ORDER BY ( ?oName)
"""

print("Results")
run_query(queryString)

Results
[('notableWork', 'http://www.wikidata.org/entity/Q1210979'), ('nwName', 'Die Aktion')]


1

In [118]:
#investigate the notable work if there is something about book
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE { 

    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5;
                wdt:P800 ?notableWork.
    ?notableWork wdt:P31 ?obj.
    ?obj ?pred ?obj1.
    
    ?obj1 <http://schema.org/name> ?oName .
} 
ORDER BY ( ?oName)
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('oName', '12 Angry Men')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('oName', '1896 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q482994'), ('oName', '19')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('oName', '1900 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('oName', '1904 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('oName', '1906 Intercalated Games')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('oName', '1908 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('oName', '1912 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('oName', '1916 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('oName', '1920 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('oName', '1924 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('oName

1547

In [121]:
#investigate the notable work if there is something about book
#I have to use the regex because there are a lot of entities called book
queryString = """
SELECT DISTINCT  ?pName ?pred ?oName ?obj1
WHERE { 

    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5;
                wdt:P800 ?notableWork.
    ?notableWork wdt:P31 ?obj.
    ?obj ?pred ?obj1.
    
    ?obj1 <http://schema.org/name> ?oName .
    ?pred <http://schema.org/name> ?pName .
    FILTER REGEX(?oName, "book").
} 
ORDER BY ( ?oName)
"""

print("Results")
run_query(queryString)

Results
[('pName', 'different from'), ('pred', 'http://www.wikidata.org/prop/direct/P1889'), ('oName', 'American comic book'), ('obj1', 'http://www.wikidata.org/entity/Q56028349')]
[('pName', "topic's main category"), ('pred', 'http://www.wikidata.org/prop/direct/P910'), ('oName', 'Category:Comic book series'), ('obj1', 'http://www.wikidata.org/entity/Q9102077')]
[('pName', "topic's main category"), ('pred', 'http://www.wikidata.org/prop/direct/P910'), ('oName', 'Category:Comic books'), ('obj1', 'http://www.wikidata.org/entity/Q9845148')]
[('pName', "topic's main category"), ('pred', 'http://www.wikidata.org/prop/direct/P910'), ('oName', 'Category:Non-fiction books'), ('obj1', 'http://www.wikidata.org/entity/Q5659271')]
[('pName', "topic's main category"), ('pred', 'http://www.wikidata.org/prop/direct/P910'), ('oName', 'Category:Role-playing game books'), ('obj1', 'http://www.wikidata.org/entity/Q8672863')]
[('pName', "topic's main category"), ('pred', 'http://www.wikidata.org/prop/dir

28

In [127]:
#return the book of notable people and their name

queryString = """
SELECT DISTINCT  ?nw ?notableWork ?notablePerson ?np
WHERE { 
    ?pub wdt:P17 wd:Q30 .
    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5;
                wdt:P800 ?notableWork.
    ?notableWork wdt:P31/wdt:P279 wd:Q571.
    
    ?notableWork <http://schema.org/name> ?nw .
    ?notablePerson <http://schema.org/name> ?np .
} 
ORDER BY ( ?oName)
"""

print("Results")
run_query(queryString)

Results
[('nw', 'The Negro Motorist Green Book'), ('notableWork', 'http://www.wikidata.org/entity/Q17114195'), ('notablePerson', 'http://www.wikidata.org/entity/Q7925963'), ('np', 'Victor Hugo Green')]
[('nw', 'Two-Fisted Tales'), ('notableWork', 'http://www.wikidata.org/entity/Q1257188'), ('notablePerson', 'http://www.wikidata.org/entity/Q732474'), ('np', 'Harvey Kurtzman')]


2

In [114]:
#investigate the notable work if there is something about book (subject)
queryString = """
SELECT DISTINCT ?sub ?sName
WHERE { 

    ?pub wdt:P31/wdt:P279* wd:Q41298.
    ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5;
                wdt:P800 ?notableWork.
    ?sub wdt:P31 ?notableWork.
    
    ?sub <http://schema.org/name> ?sName .
} 
ORDER BY DESC( ?sName)
"""

print("Results")
run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q76366341'), ('sName', 'William Ramsay Maule, 1st Baron Panmure')]
[('sub', 'http://www.wikidata.org/entity/Q76366858'), ('sName', 'Welsh goat in danger')]
[('sub', 'http://www.wikidata.org/entity/Q76366678'), ('sName', 'View of Leith Races on Saturday 26 July 1800')]
[('sub', 'http://www.wikidata.org/entity/Q11705008'), ('sName', 'Vath Sarn')]
[('sub', 'http://www.wikidata.org/entity/Q76366897'), ('sName', 'Unidentified woman')]
[('sub', 'http://www.wikidata.org/entity/Q76366604'), ('sName', 'Unidentified soldiers on horseback')]
[('sub', 'http://www.wikidata.org/entity/Q76366413'), ('sName', 'Unidentified soldier')]
[('sub', 'http://www.wikidata.org/entity/Q76366296'), ('sName', 'Unidentified soldier')]
[('sub', 'http://www.wikidata.org/entity/Q76366500'), ('sName', 'Unidentified man')]
[('sub', 'http://www.wikidata.org/entity/Q76366862'), ('sName', 'Unidentified man')]
[('sub', 'http://www.wikidata.org/entity/Q45715190'), ('sName', 'U

218

******
# Query 4: PART 1
Get the name of notable employees working for The New Yorker and any other political magazine published in the US. Check if these employees have witten any book and if so get the title. 

In [130]:
#return the notable employees connected to politicians publication published in US (+ The New Yorker)
#First BGP: only The New Yorker
#Second BGP: politicians publication published in US
#NB: I've considered the employees that have done some notable works as notable employees
queryString = """
SELECT DISTINCT ?work ?person
WHERE { 
    {
        ?notablePerson wdt:P800 wd:Q217305;
                wdt:P31 wd:Q5;
                wdt:P800 ?notableWork.
    }
    UNION
    {
        ?pub wdt:P17 wd:Q30 .
        ?pub wdt:P31/wdt:P279* wd:Q41298.
        ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5;
                wdt:P800 ?notableWork.             
    }
    
    ?notableWork <http://schema.org/name> ?work .
    ?notablePerson <http://schema.org/name> ?person .
} 
ORDER BY DESC( ?np)
"""

print("Results")
run_query(queryString)

Results
[('work', 'The New Yorker'), ('person', 'Loveis Wise')]
[('work', 'SAGE Publications'), ('person', 'George D. McCune')]
[('work', 'SAGE Publications'), ('person', 'Sara Miller McCune')]
[('work', 'Elsevier BV'), ('person', 'Jacobus George Robbers')]
[('work', 'Winkler Prins'), ('person', 'Jacobus George Robbers')]
[('work', "Elsevier's Geïllustreerd Maandschrift"), ('person', 'Jacobus George Robbers')]
[('work', 'The Astronomical Journal'), ('person', 'Benjamin Apthorp Gould')]
[('work', 'The Cross and the Flag'), ('person', 'Gerald L. K. Smith')]
[('work', 'Turn Me On'), ('person', 'Beverly Afaglo')]
[('work', 'Popular Science'), ('person', 'Waldemar Kaempffert')]
[('work', 'The New York Review of Books'), ('person', 'David Levine')]
[('work', "Asimov's Science Fiction"), ('person', 'Gardner Dozois')]
[('work', "Asimov's Science Fiction"), ('person', 'George H. Scithers')]
[('work', 'Playboy'), ('person', 'Beverly Afaglo')]
[('work', 'Playboy'), ('person', 'Hugh Hefner')]
[('w

42

# Query 4: Part 2

In [131]:
#return the notable employees connected to politicians publication published in US (+ The New Yorker)
#First: only The New Yorker
#Second: politicians publication published in US
#NB: I've considered the employees that have done some notable works as notable employees
queryString = """
SELECT DISTINCT ?work ?person 
WHERE { 
    {
        ?notablePerson wdt:P800 wd:Q217305;
                wdt:P31 wd:Q5;
                wdt:P800 ?notableWork.
    }
    UNION
    {
        ?pub wdt:P17 wd:Q30 .
        ?pub wdt:P31/wdt:P279* wd:Q41298.
        ?notablePerson wdt:P800 ?pub;
                wdt:P31 wd:Q5;
                wdt:P800 ?notableWork.             
    }
    ?notableWork wdt:P31/wdt:P279 wd:Q571.
    ?notableWork <http://schema.org/name> ?work .
    ?notablePerson <http://schema.org/name> ?person .
} 
ORDER BY DESC( ?np)
"""

print("Results")
run_query(queryString)

Results
[('work', 'The Negro Motorist Green Book'), ('person', 'Victor Hugo Green')]
[('work', 'Two-Fisted Tales'), ('person', 'Harvey Kurtzman')]


2

> During the task, I did a mistake because I've confused 'employer' for 'employees'.
>
> So, I've searched for the notableWork and then from this I've searched something with book.
***********

****************
# Query 5

In [71]:
#ask if there are some employees connected to The New Yorker that has ever been nominated for an award
queryString = """
ASK WHERE {
    SELECT DISTINCT ?employeesName ?obj
    WHERE { 
        wd:Q217305 wdt:P31/wdt:P279* wd:Q41298.
        ?employees wdt:P108 wd:Q217305.
        ?employees wdt:P1411 ?award.


        ?employees <http://schema.org/name> ?employeesName .
}
} 
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

> I've used the information found during the other queries.
****************